In [1]:
from utils.files import FileManager
import pandas as pd

inputs_fm = FileManager('../inputs/')
outputs_fm = FileManager('../outputs/')

In [24]:
drugs_df = inputs_fm.load('drugs.csv')
pubmed_df = inputs_fm.load('pubmed.csv', parse_dates=['date'], infer_datetime_format=True)
trials_df = inputs_fm.load('clinical_trials.csv', parse_dates=['date'], infer_datetime_format=True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/io/parsers/base_parser.py:1070: UserWarning: Parsing '25/05/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(


In [3]:
drugs_df.head()

,atccode,drug
0,A04AD,DIPHENHYDRAMINE
1,S03AA,TETRACYCLINE
2,V03AB,ETHANOL
3,A03BA,ATROPINE
4,A01AD,EPINEPHRINE


In [4]:
pubmed_df.head()

,id,title,date,journal
0,1,A 44-year-old man with erythema of the face di...,2019-01-01,Journal of emergency nursing
1,2,"An evaluation of benadryl, pyribenzamine, and ...",2019-01-01,Journal of emergency nursing
2,3,Diphenhydramine hydrochloride helps symptoms o...,2019-02-01,The Journal of pediatrics
3,4,Tetracycline Resistance Patterns of Lactobacil...,2020-01-01,Journal of food protection
4,5,Appositional Tetracycline bone formation rates...,2020-02-01,American journal of veterinary research


In [5]:
trials_df.head()

,id,scientific_title,date,journal
0,NCT01967433,Use of Diphenhydramine as an Adjunctive Sedati...,2020-01-01,Journal of emergency nursing
1,NCT04189588,Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydroch...,2020-01-01,Journal of emergency nursing
2,NCT04237090,,2020-01-01,Journal of emergency nursing
3,NCT04237091,Feasibility of a Randomized Controlled Clinica...,2020-01-01,Journal of emergency nursing
4,NCT04153396,Preemptive Infiltration With Betamethasone and...,2020-01-01,Hôpitaux Universitaires de Genève


In [8]:
import re

all_drugs_regex = "|".join(drugs_df['drug'])

found_drugs_ss = pubmed_df['title'].str.findall(all_drugs_regex, re.IGNORECASE).explode().str.upper().dropna()
found_drugs_ss.name = 'drug'

pubmed_with_drugs_df = pd.merge(pubmed_df, found_drugs_ss, left_index=True, right_index=True)
outputs_fm.save(pubmed_with_drugs_df[['id', 'date', 'drug']], 'tmp/pubmed.csv', index=False)
pubmed_with_drugs_df.head()

,id,title,date,journal,drug
0,1,A 44-year-old man with erythema of the face di...,2019-01-01,Journal of emergency nursing,DIPHENHYDRAMINE
1,2,"An evaluation of benadryl, pyribenzamine, and ...",2019-01-01,Journal of emergency nursing,DIPHENHYDRAMINE
2,3,Diphenhydramine hydrochloride helps symptoms o...,2019-02-01,The Journal of pediatrics,DIPHENHYDRAMINE
3,4,Tetracycline Resistance Patterns of Lactobacil...,2020-01-01,Journal of food protection,TETRACYCLINE
4,5,Appositional Tetracycline bone formation rates...,2020-02-01,American journal of veterinary research,TETRACYCLINE


In [10]:
trials_df = trials_df.rename(columns={'scientific_title': 'title'})

found_drugs_ss = trials_df['title'].str.findall(all_drugs_regex, re.IGNORECASE).explode().str.upper().dropna()
found_drugs_ss.name = 'drug'

trials_with_drugs_df = pd.merge(trials_df, found_drugs_ss, left_index=True, right_index=True)
outputs_fm.save(trials_with_drugs_df[['id', 'date', 'drug']], 'tmp/clinical_trial.csv', index=False)
trials_with_drugs_df.head()

,id,title,date,journal,drug
0,NCT01967433,Use of Diphenhydramine as an Adjunctive Sedati...,2020-01-01,Journal of emergency nursing,DIPHENHYDRAMINE
1,NCT04189588,Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydroch...,2020-01-01,Journal of emergency nursing,DIPHENHYDRAMINE
3,NCT04237091,Feasibility of a Randomized Controlled Clinica...,2020-01-01,Journal of emergency nursing,DIPHENHYDRAMINE
4,NCT04153396,Preemptive Infiltration With Betamethasone and...,2020-01-01,Hôpitaux Universitaires de Genève,BETAMETHASONE
7,NCT04188184,Tranexamic Acid Versus Epinephrine During Expl...,2020-04-27,Journal of emergency nursing\xc3\x28,EPINEPHRINE


In [14]:
drug_mention_dag = pd.merge(drugs_df, outputs_fm.load('tmp/pubmed.csv'), on='drug')
drug_mention_dag['source'] = 'pubmed'
drug_mention_dag.head()

,atccode,drug,id,date,source
0,A04AD,DIPHENHYDRAMINE,1,2019-01-01,pubmed
1,A04AD,DIPHENHYDRAMINE,2,2019-01-01,pubmed
2,A04AD,DIPHENHYDRAMINE,3,2019-02-01,pubmed
3,S03AA,TETRACYCLINE,4,2020-01-01,pubmed
4,S03AA,TETRACYCLINE,5,2020-02-01,pubmed


In [17]:
drug_mention_dag = pd.merge(drugs_df, outputs_fm.load('tmp/clinical_trial.csv'), on='drug')
drug_mention_dag['source'] = 'clinical_trial'
drug_mention_dag.head()

,atccode,drug,id,date,source
0,A04AD,DIPHENHYDRAMINE,NCT01967433,2020-01-01,clinical_trial
1,A04AD,DIPHENHYDRAMINE,NCT04189588,2020-01-01,clinical_trial
2,A04AD,DIPHENHYDRAMINE,NCT04237091,2020-01-01,clinical_trial
3,A01AD,EPINEPHRINE,NCT04188184,2020-04-27,clinical_trial
4,R01AD,BETAMETHASONE,NCT04153396,2020-01-01,clinical_trial


In [19]:
mentions = pd.concat([outputs_fm.load('tmp/pubmed.csv'), outputs_fm.load('tmp/clinical_trial.csv')])
mentions.head(n=100)

,id,date,drug
0,1,2019-01-01,DIPHENHYDRAMINE
1,2,2019-01-01,DIPHENHYDRAMINE
2,3,2019-02-01,DIPHENHYDRAMINE
3,4,2020-01-01,TETRACYCLINE
4,5,2020-02-01,TETRACYCLINE
5,6,2020-01-01,TETRACYCLINE
6,6,2020-01-01,ETHANOL
7,6,2020-01-01,ETHANOL
8,7,2020-01-02,EPINEPHRINE
9,8,2020-01-03,EPINEPHRINE


In [14]:
dag_df = outputs_fm.load('dag.json', orient="records")
dag_df.head()

,id,date,drug,atccode,source
0,NCT01967433,2020-01-01,DIPHENHYDRAMINE,A04AD,clinical_trials
1,NCT04189588,2020-01-01,DIPHENHYDRAMINE,A04AD,clinical_trials
2,NCT04237091,2020-01-01,DIPHENHYDRAMINE,A04AD,clinical_trials
3,NCT04153396,2020-01-01,BETAMETHASONE,R01AD,clinical_trials
4,NCT04188184,2020-04-27,EPINEPHRINE,A01AD,clinical_trials


In [19]:
dag_df.groupby('id').count()

date       3
drug       3
atccode    3
source     3
dtype: int64